get and format dataset

In [67]:
# load in wine 
import pandas as pd

# Define column names
column_names = [
    "Class", "Alcohol", "Malicacid", "Ash", "Alcalinity_of_ash", "Magnesium",
    "Total_phenols", "Flavanoids", "Nonflavanoid_phenols", "Proanthocyanins",
    "Color_intensity", "Hue", "OD280_OD315_of_diluted_wines", "Proline"
]

# Load the CSV file
df = pd.read_csv("wine/wine.data", names=column_names)

# Display the first few rows
print(df.head())

   Class  Alcohol  Malicacid   Ash  Alcalinity_of_ash  Magnesium  \
0      1    14.23       1.71  2.43               15.6        127   
1      1    13.20       1.78  2.14               11.2        100   
2      1    13.16       2.36  2.67               18.6        101   
3      1    14.37       1.95  2.50               16.8        113   
4      1    13.24       2.59  2.87               21.0        118   

   Total_phenols  Flavanoids  Nonflavanoid_phenols  Proanthocyanins  \
0           2.80        3.06                  0.28             2.29   
1           2.65        2.76                  0.26             1.28   
2           2.80        3.24                  0.30             2.81   
3           3.85        3.49                  0.24             2.18   
4           2.80        2.69                  0.39             1.82   

   Color_intensity   Hue  OD280_OD315_of_diluted_wines  Proline  
0             5.64  1.04                          3.92     1065  
1             4.38  1.05        

In [68]:
# Remove rows where Class is 3
df = df[df["Class"] != 3]

# Display the first few rows to verify
print(df.tail())

     Class  Alcohol  Malicacid   Ash  Alcalinity_of_ash  Magnesium  \
125      2    12.07       2.16  2.17               21.0         85   
126      2    12.43       1.53  2.29               21.5         86   
127      2    11.79       2.13  2.78               28.5         92   
128      2    12.37       1.63  2.30               24.5         88   
129      2    12.04       4.30  2.38               22.0         80   

     Total_phenols  Flavanoids  Nonflavanoid_phenols  Proanthocyanins  \
125           2.60        2.65                  0.37             1.35   
126           2.74        3.15                  0.39             1.77   
127           2.13        2.24                  0.58             1.76   
128           2.22        2.45                  0.40             1.90   
129           2.10        1.75                  0.42             1.35   

     Color_intensity   Hue  OD280_OD315_of_diluted_wines  Proline  
125             2.76  0.86                          3.28      378  
126 

Implement a logistic regression baseline

In [69]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [70]:
y = df['Class']
X = df.drop(columns=['Class'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [71]:
model = LogisticRegression(max_iter=700)
model.fit(X_train, y_train)

LogisticRegression(max_iter=700)

In [72]:
y_pred = model.predict(X_test)

# Evaluate performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Baseline Logistic Regression Accuracy: {accuracy:.4f}")

Baseline Logistic Regression Accuracy: 1.0000


Now, we specify a custom method of selecting the next coordinate. For this, lets create a function that takes in X and y, and returns an array specifying the order we want to test in. 

Solution idea: Cluster the data by features, and then rotate data points from each cluster. 

In [73]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

Now we need a custom class so we can directly modify the loss function... 

In [74]:
import numpy as np
from scipy.special import expit
from itertools import product
import multiprocessing
from joblib import Parallel, delayed

class LogisticRegression:
    def __init__(self, lr=0.001, max_iter=2000, tol=1e-4, fit_intercept=True, reg_lambda=0.01):
        self.lr = lr
        self.max_iter = max_iter
        self.tol = tol  # Tolerance for weight convergence
        self.fit_intercept = fit_intercept
        self.reg_lambda = reg_lambda
        self.weight = None
        self.feature_means = None
        self.feature_stds = None
        self.convergence_window = 10  # Number of iterations to check for weight stability

    def _normalize_features(self, X):
        """Normalizes features using z-score standardization."""
        if self.feature_means is None or self.feature_stds is None:
            self.feature_means = np.mean(X, axis=0)
            self.feature_stds = np.std(X, axis=0) + 1e-8  # Avoid division by zero
        
        return (X - self.feature_means) / self.feature_stds

    def _add_intercept(self, X):
        """Adds intercept term (bias) to feature matrix."""
        return np.c_[np.ones((X.shape[0], 1)), X]

    def _sigmoid(self, z):
        """Sigmoid activation function."""
        # Clip values to prevent overflow
        z = np.clip(z, -20, 20)
        return expit(z)  # Numerically stable sigmoid

    def _compute_loss(self, X, y):
        """Computes the binary cross-entropy loss with regularization."""
        m = len(y)
        h = self._sigmoid(X @ self.weight)
        # Add small epsilon to avoid log(0)
        epsilon = 1e-15
        h = np.clip(h, epsilon, 1 - epsilon)
        
        # Add L2 regularization term (excluding bias)
        reg_term = 0.5 * self.reg_lambda * np.sum(self.weight[1:] ** 2) / m
        
        # Compute binary cross-entropy
        loss = -np.mean(y * np.log(h) + (1 - y) * np.log(1 - h))
        
        return loss + reg_term
    
    def _compute_gradient_for_coordinate(self, X, y, j, sample_indices=None, sample_weights=None):
        """
        Computes the gradient for a single coordinate j using only the specified samples.
        
        Parameters:
        -----------
        X : array-like
            Feature matrix
        y : array-like
            Target values
        j : int
            Index of the coordinate to compute gradient for
        sample_indices : array-like or None
            Indices of samples to use for gradient computation. If None, use all samples.
        sample_weights : array-like or None
            Weights for each sample. If None, equal weights are used.
        """
        if sample_indices is not None:
            X_batch = X[sample_indices]
            y_batch = y[sample_indices]
            if sample_weights is not None:
                weights_batch = sample_weights[sample_indices]
            else:
                weights_batch = None
        else:
            X_batch = X
            y_batch = y
            weights_batch = sample_weights
        
        m = len(y_batch)
        h = self._sigmoid(X_batch @ self.weight)
        
        if weights_batch is None:
            weights_batch = np.ones(m)
            
        # Gradient for single coordinate using only the batch
        gradient_j = np.sum((h - y_batch) * weights_batch * X_batch[:, j]) / np.sum(weights_batch)
        
        # Add L2 regularization (except for bias term)
        if j > 0 or not self.fit_intercept:
            gradient_j += self.reg_lambda * self.weight[j] / m
            
        return gradient_j
    
    def _custom_gradient_for_coordinate(self, X, y, j, sample_indices=None, sample_weights=None):
        """
        Custom gradient computation for a single coordinate.
        This implements a focal loss variant that puts more emphasis on hard examples.
        
        Parameters:
        -----------
        X : array-like
            Feature matrix
        y : array-like
            Target values
        j : int
            Index of the coordinate to compute gradient for
        sample_indices : array-like or None
            Indices of samples to use for gradient computation
        sample_weights : array-like or None
            Weights for each sample
        """
        if sample_indices is not None:
            X_batch = X[sample_indices]
            y_batch = y[sample_indices]
            if sample_weights is not None:
                weights_batch = sample_weights[sample_indices]
            else:
                weights_batch = None
        else:
            X_batch = X
            y_batch = y
            weights_batch = sample_weights
        
        m = len(y_batch)
        
        # Convert labels from 1/2 to 0/1 for easier calculation
        y_binary = (y_batch == 2).astype(int)
        
        # Get current predictions
        h = self._sigmoid(X_batch @ self.weight)
        
        # Calculate errors with focal loss modulation
        # Focal loss puts more weight on hard examples
        gamma = 2.0  # Focusing parameter
        
        # pt is the probability of the true class
        pt = h * y_binary + (1 - h) * (1 - y_binary)
        # Focal weight is (1-pt)^gamma
        focal_weights = np.power(1 - pt, gamma)
        
        if weights_batch is None:
            weights_batch = np.ones(m)
            
        # Combine focal weights with sample weights
        combined_weights = weights_batch * focal_weights
        
        # Gradient for single coordinate using focal loss
        errors = h - y_binary
        gradient_j = np.sum(errors * combined_weights * X_batch[:, j]) / np.sum(combined_weights)
        
        # Add L2 regularization (except for bias term)
        if j > 0 or not self.fit_intercept:
            gradient_j += self.reg_lambda * self.weight[j] / m
            
        return gradient_j

    def _get_coordinate(self, X, y, n_features, method="random", sample_weights=None):
        """
        Determine the next coordinate to update for stochastic coordinate descent.
        
        Parameters:
        -----------
        X : array-like
            Feature matrix
        y : array-like
            Target values
        n_features : int
            Number of features
        method : str
            Method to select coordinates ("random", "gradient_based", "cyclic", 
            "cyclic_momentum", "importance_sampling", "greedy_subset", "adaptive_frequency")
        sample_weights : array-like or None
            Weights for each sample
            
        Returns:
        --------
        int: Index of the feature (coordinate) to update
        """
        if method == "random":
            # Randomly select a coordinate
            return np.random.randint(0, n_features)
        
        elif method == "gradient_based":
            # Select coordinate based on gradient magnitude
            gradients = np.zeros(n_features)
            for j in range(n_features):
                gradients[j] = abs(self._compute_gradient_for_coordinate(X, y, j, sample_weights=sample_weights))
            
            # Select coordinate with highest gradient magnitude
            return np.argmax(gradients)
        
        elif method == "cyclic":
            # Use the class attribute to keep track of the current coordinate
            if not hasattr(self, '_current_coordinate'):
                self._current_coordinate = 0
            else:
                self._current_coordinate = (self._current_coordinate + 1) % n_features
            
            return self._current_coordinate
        
        elif method == "cyclic_momentum":
            # Cyclic coordinate descent with momentum
            if not hasattr(self, '_current_coordinate'):
                self._current_coordinate = 0
                self._momentum = np.zeros(n_features)
            else:
                self._current_coordinate = (self._current_coordinate + 1) % n_features
            
            # Store current gradient for momentum calculation in the update step
            current_grad = self._compute_gradient_for_coordinate(X, y, self._current_coordinate, sample_weights=sample_weights)
            
            # Add momentum to the update (will be used in fit method)
            momentum_beta = 0.9
            if not hasattr(self, '_gradient_cache'):
                self._gradient_cache = {}
            self._gradient_cache[self._current_coordinate] = current_grad
            
            if not hasattr(self, '_momentum'):
                self._momentum = np.zeros(n_features)
            
            self._momentum[self._current_coordinate] = momentum_beta * self._momentum[self._current_coordinate] + \
                (1 - momentum_beta) * current_grad
            
            return self._current_coordinate
        
        elif method == "importance_sampling":
            # Select coordinates with probability proportional to historical gradient magnitudes
            if not hasattr(self, '_gradient_history'):
                self._gradient_history = np.ones(n_features)
            
            # Get a sample of gradients to update history
            sample_size = min(5, n_features)
            sample_coords = np.random.choice(n_features, sample_size, replace=False)
            
            for j in sample_coords:
                grad_j = abs(self._compute_gradient_for_coordinate(X, y, j, sample_weights=sample_weights))
                self._gradient_history[j] = 0.9 * self._gradient_history[j] + 0.1 * grad_j
            
            # Add small constant to avoid zero probabilities
            probs = self._gradient_history + 1e-5
            probs = probs / np.sum(probs)
            
            # Sample coordinate based on probabilities
            coordinate = np.random.choice(n_features, p=probs)
            
            return coordinate
        
        elif method == "greedy_subset":
            # Compute gradients for a small random subset and pick largest
            subset_size = min(5, n_features)
            coordinate_subset = np.random.choice(n_features, subset_size, replace=False)
            
            max_grad = 0
            best_coordinate = coordinate_subset[0]  # Default to first in case all gradients are 0
            
            for j in coordinate_subset:
                grad_j = abs(self._compute_gradient_for_coordinate(X, y, j, sample_weights=sample_weights))
                if grad_j > max_grad:
                    max_grad = grad_j
                    best_coordinate = j
                    
            return best_coordinate
        
        elif method == "adaptive_frequency":
            # Update more frequently coordinates with historically larger gradient changes
            if not hasattr(self, '_update_frequencies'):
                self._update_frequencies = np.ones(n_features)
                self._last_gradients = np.zeros(n_features)
            
            # Sample a coordinate to evaluate
            if np.random.random() < 0.1:  # Exploration rate
                coordinate = np.random.randint(0, n_features)
            else:
                # Normalize frequencies to get probabilities
                scores = self._update_frequencies / np.sum(self._update_frequencies)
                coordinate = np.random.choice(n_features, p=scores)
            
            # Calculate current gradient for this coordinate
            current_grad = self._compute_gradient_for_coordinate(X, y, coordinate, sample_weights=sample_weights)
            
            # Update frequencies based on gradient changes
            grad_change = abs(current_grad - self._last_gradients[coordinate])
            self._update_frequencies[coordinate] *= (1.0 + grad_change)
            self._last_gradients[coordinate] = current_grad
            
            return coordinate
        
        else:
            # Default to random if invalid method
            return np.random.randint(0, n_features)
    
    def _handle_class_imbalance(self, X, y):
        """Calculate class weights for imbalanced datasets."""
        unique_classes = np.unique(y)
        class_weights = {}
        
        for cls in unique_classes:
            class_weights[cls] = len(y) / (len(unique_classes) * np.sum(y == cls))
            
        # Create sample weights array
        sample_weights = np.ones(len(y))
        for cls in unique_classes:
            sample_weights[y == cls] = class_weights[cls]
            
        return sample_weights
    
    def fit(self, X, y, loss_type="baseline", coordinate_choice="random", 
        batch_size=1, handle_imbalance=True, verbose=True):
        """
        Trains the logistic regression model using stochastic coordinate descent.
        
        Parameters:
        -----------
        X : array-like, shape (n_samples, n_features)
            Training data
        y : array-like, shape (n_samples,)
            Target values
        loss_type : str, default='baseline'
            Loss function type ('baseline' or 'custom')
        coordinate_choice : str, default='random'
            How to choose coordinates for SGD ('random', 'gradient_based', or 'cyclic')
        batch_size : int, default=1
            Number of samples to use in each update (true SGD uses 1)
        handle_imbalance : bool, default=True
            Whether to weigh samples by inverse class frequency
        verbose : bool, default=True
            Whether to print progress
            
        Returns:
        --------
        accuracies : array-like
            Array of accuracy values during training
        """
        # Convert inputs to numpy arrays
        X = np.asarray(X)
        y = np.asarray(y)
        
        # Normalize features
        X_normalized = self._normalize_features(X)
        
        if self.fit_intercept:
            X_normalized = self._add_intercept(X_normalized)
    
        # Initialize weights with small random values
        np.random.seed(42)  # For reproducibility
        n_features = X_normalized.shape[1]
        self.weight = np.random.randn(n_features) * 0.01
        
        # Handle class imbalance
        if handle_imbalance:
            sample_weights = self._handle_class_imbalance(X, y)
        else:
            sample_weights = np.ones(len(y))
        
        # Store weight history for convergence check
        weight_history = [np.copy(self.weight)]
        converged = False
        
        # Array to track accuracies during training
        accuracies = []
        check_interval = max(1, self.max_iter // 100)  # Check accuracy ~100 times
        
        min_updates_per_coordinate = 3  # Ensure each coordinate has been updated enough times
        coordinate_update_counts = np.zeros(n_features)
        
        n_samples = X.shape[0]
        
        for i in range(self.max_iter):
            # Get the next coordinate
            j = self._get_coordinate(
                X_normalized, y, n_features, method=coordinate_choice, sample_weights=sample_weights
            )
            
            # For true SGD, use a single random sample
            if batch_size == 1:
                batch_indices = [np.random.choice(n_samples)]
            else:
                batch_indices = np.random.choice(n_samples, size=batch_size)
            
            # Track coordinate updates
            coordinate_update_counts[j] += 1
            
            # Use appropriate gradient calculation for the selected coordinate
            if loss_type == "baseline":
                gradient_j = self._compute_gradient_for_coordinate(
                    X_normalized, y, j, batch_indices, sample_weights
                )
            else:  # Custom loss
                gradient_j = self._custom_gradient_for_coordinate(
                    X_normalized, y, j, batch_indices, sample_weights
                )
            
            # Update only the selected coordinate
            self.weight[j] -= self.lr * gradient_j
            
            # Check convergence and track accuracy periodically
            if i % check_interval == 0 or i == self.max_iter - 1:
                # Store current weights for convergence check
                weight_history.append(np.copy(self.weight))
                
                # Keep only the most recent weights for convergence check
                if len(weight_history) > self.convergence_window:
                    weight_history.pop(0)
                
                # Calculate current accuracy
                y_pred = (self._sigmoid(X_normalized @ self.weight) >= 0.5).astype(int) + 1
                current_accuracy = np.mean(y_pred == y)
                accuracies.append(current_accuracy)
                
                # Check for weight convergence if we have enough history
                if len(weight_history) == self.convergence_window:
                    # Check if all coordinates have been updated enough times
                    if np.all(coordinate_update_counts >= min_updates_per_coordinate):
                        # Calculate max weight change across the window and all coordinates
                        weight_changes = []
                        for w_idx in range(1, len(weight_history)):
                            # Calculate absolute change in weights
                            weight_diff = np.abs(weight_history[w_idx] - weight_history[w_idx-1])
                            # Get maximum change for any coordinate
                            weight_changes.append(np.max(weight_diff))
                        
                        # Check if max weight change is less than tolerance
                        if np.max(weight_changes) < self.tol:
                            if verbose:
                                print(f'Converged at iteration {i}, weights stabilized with max change: {np.max(weight_changes):.6f}, accuracy: {current_accuracy:.4f}')
                            converged = True
                            # Fill the rest of the accuracy array with the final value
                            accuracies.extend([current_accuracy] * ((self.max_iter // check_interval) - len(accuracies)))
                            break
                
                # Adaptive learning rate
                if i > 0 and i % (5 * n_features) == 0:
                    self.lr *= 0.90
        
        if not converged and verbose:
            print(f'Did not converge after {self.max_iter} iterations. Final accuracy: {current_accuracy:.4f}')
            
        # Make sure we return exactly 100 points for easier plotting
        if len(accuracies) < 100:
            # Repeat last value to fill
            accuracies.extend([accuracies[-1]] * (100 - len(accuracies)))
        elif len(accuracies) > 100:
            # Downsample by taking evenly spaced points
            indices = np.linspace(0, len(accuracies)-1, 100, dtype=int)
            accuracies = [accuracies[i] for i in indices]
            
        return np.array(accuracies)

    def predict_proba(self, X):
        """Returns the probability predictions."""
        X = np.asarray(X)
        X_normalized = self._normalize_features(X)
        
        if self.fit_intercept:
            X_normalized = self._add_intercept(X_normalized)

        return self._sigmoid(X_normalized @ self.weight)

    def predict(self, X, threshold=0.5):
        """Predicts class labels (0 or 1) based on a threshold."""
        return (self.predict_proba(X) >= threshold).astype(int) + 1
    
    def score(self, X, y):
        """Calculate accuracy score."""
        y_pred = self.predict(X)
        return np.mean(y_pred == y)
    
    def get_feature_importance(self):
        """Returns the absolute weights as feature importance."""
        if self.fit_intercept:
            return np.abs(self.weight[1:])
        return np.abs(self.weight)

In [75]:
def grid_search(X, y, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=True, 
               loss_type="baseline", coordinate_choice="random"):
    """
    Perform grid search to find optimal hyperparameters.
    
    Parameters:
    -----------
    X : array-like, shape (n_samples, n_features)
        Training data
    y : array-like, shape (n_samples,)
        Target values
    param_grid : dict
        Dictionary with parameter names as keys and lists of parameter values
    cv : int, default=5
        Number of cross-validation folds
    scoring : str, default='accuracy'
        Scoring metric ('accuracy', 'precision', 'recall', 'f1', 'auc')
    n_jobs : int, default=-1
        Number of jobs to run in parallel (-1 means using all processors)
    verbose : bool, default=True
        Whether to print progress
    loss_type : str, default='baseline'
        Loss function type ('baseline' or 'custom')
    coordinate_choice : str, default='random'
        How to choose coordinates for SGD ('random', 'gradient_based', or 'cyclic')
        
    Returns:
    --------
    dict : Best parameters, corresponding score, and convergence curves
    """
    # Convert inputs to numpy arrays if they're not already
    X = np.asarray(X)
    y = np.asarray(y)
    
    # Prepare parameter combinations
    param_names = list(param_grid.keys())
    param_values = list(param_grid.values())
    param_combinations = list(product(*param_values))
    
    # Split data into folds
    n_samples = len(y)
    indices = np.arange(n_samples)
    np.random.seed(42)  # For reproducible CV splits
    np.random.shuffle(indices)
    fold_sizes = np.full(cv, n_samples // cv, dtype=int)
    fold_sizes[:n_samples % cv] += 1
    
    current_idx = 0
    folds = []
    for fold_size in fold_sizes:
        fold_indices = indices[current_idx:current_idx + fold_size]
        folds.append(fold_indices)
        current_idx += fold_size
        
    # Define evaluation function for a single parameter combination
    def evaluate_params(params):
        param_dict = {param_names[i]: params[i] for i in range(len(param_names))}
        
        if verbose:
            print(f"Evaluating parameters: {param_dict}")
        
        scores = []
        convergence_curves = []
        
        for i in range(cv):
            # Split data
            test_idx = folds[i]
            train_idx = np.concatenate([folds[j] for j in range(cv) if j != i])
            
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]
            
            # Train model with current parameters
            model = LogisticRegression(**param_dict)
            
            # Use specified coordinate selection and loss type
            # Store accuracy history from fit method
            accuracy_history = model.fit(
                X_train, y_train,
                loss_type=loss_type,
                coordinate_choice=coordinate_choice,
                verbose=False
            )
            
            # Store convergence curve for this fold
            convergence_curves.append(accuracy_history)
            
            # Evaluate based on scoring metric
            if scoring == 'accuracy':
                score = model.score(X_test, y_test)
            elif scoring == 'precision':
                y_pred = model.predict(X_test)
                score = np.sum((y_pred == 2) & (y_test == 2)) / np.sum(y_pred == 2) if np.sum(y_pred == 2) > 0 else 0
            elif scoring == 'recall':
                y_pred = model.predict(X_test)
                score = np.sum((y_pred == 2) & (y_test == 2)) / np.sum(y_test == 2) if np.sum(y_test == 2) > 0 else 0
            elif scoring == 'f1':
                y_pred = model.predict(X_test)
                precision = np.sum((y_pred == 2) & (y_test == 2)) / np.sum(y_pred == 2) if np.sum(y_pred == 2) > 0 else 0
                recall = np.sum((y_pred == 2) & (y_test == 2)) / np.sum(y_test == 2) if np.sum(y_test == 2) > 0 else 0
                score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
            elif scoring == 'auc':
                try:
                    from sklearn.metrics import roc_auc_score
                    y_prob = model.predict_proba(X_test)
                    score = roc_auc_score(y_test == 2, y_prob)
                except ImportError:
                    # Fallback if sklearn is not available
                    y_prob = model.predict_proba(X_test)
                    # Manual AUC calculation (simplified)
                    pos_scores = y_prob[y_test == 2]
                    neg_scores = y_prob[y_test != 2]
                    if len(pos_scores) == 0 or len(neg_scores) == 0:
                        score = 0.5
                    else:
                        n_pos = len(pos_scores)
                        n_neg = len(neg_scores)
                        score = sum(pos > neg for pos in pos_scores for neg in neg_scores) / (n_pos * n_neg)
            else:
                raise ValueError(f"Unknown scoring metric: {scoring}")
            
            scores.append(score)
        
        mean_score = np.mean(scores)
        if verbose:
            print(f"Parameters {param_dict} - Mean {scoring}: {mean_score:.4f}")
            # Check if model converged in all folds
            avg_iterations = np.mean([len(np.unique(curve)) for curve in convergence_curves])
            print(f"Average unique accuracy points: {avg_iterations:.1f}/100 (higher suggests better convergence)")
        
        # Average convergence curve across folds
        avg_convergence = np.mean(convergence_curves, axis=0)
        
        return param_dict, mean_score, avg_convergence
    
    # For small grid sizes or when parallelism causes issues, run sequentially
    if len(param_combinations) <= 4 or n_jobs == 1:
        results = [evaluate_params(params) for params in param_combinations]
    else:
        # Run evaluations in parallel
        try:
            n_jobs = n_jobs if n_jobs > 0 else multiprocessing.cpu_count()
            results = Parallel(n_jobs=n_jobs)(
                delayed(evaluate_params)(params) for params in param_combinations
            )
        except Exception as e:
            if verbose:
                print(f"Parallel execution failed with error: {str(e)}")
                print("Falling back to sequential execution...")
            results = [evaluate_params(params) for params in param_combinations]
    
    # Find best parameters
    best_params, best_score, best_convergence = max(results, key=lambda x: x[1])
    
    if verbose:
        opt_method = f"Coordinate descent with {coordinate_choice} selection"
        print(f"\nGrid Search Results (using {opt_method}):")
        print(f"Best parameters: {best_params}")
        print(f"Best {scoring}: {best_score:.4f}")
        
        # Calculate iterations until convergence for best model
        # Find point where accuracy stabilizes (change < 0.001)
        diffs = np.abs(np.diff(best_convergence))
        stable_idx = np.argmax(diffs < 0.001)
        if stable_idx > 0:
            print(f"Best model converged after ~{stable_idx} iterations")
        else:
            print("Best model may not have fully converged")
    
    return {
        'best_params': best_params,
        'best_score': best_score,
        'best_convergence': best_convergence,
        'all_results': [(params, score) for params, score, _ in results],
        'convergence_curves': {str(params): curve for params, _, curve in results}
    }

In [76]:
def fullBreakdown(X, y, param_grid=None, cv=5, n_jobs=-1):
    """
    Runs a comprehensive analysis of the LogisticRegression model:
    1. Finds optimal hyperparameters using grid search
    2. Compares baseline vs custom loss with random coordinate selection
    3. Compares different coordinate selection methods with the custom loss
    
    Parameters:
    -----------
    X : array-like
        Feature matrix
    y : array-like
        Target values
    param_grid : dict, optional
        Parameter grid for grid search (default provides a basic grid)
    cv : int, default=5
        Number of cross-validation folds
    n_jobs : int, default=-1
        Number of parallel jobs
        
    Returns:
    --------
    dict
        Results of the analysis
    """
    
    # Convert inputs to numpy arrays
    X = np.asarray(X)
    y = np.asarray(y)
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    # Default parameter grid if none provided
    if param_grid is None:
        param_grid = {
            'lr': [0.001, 0.01, 0.1],
            'max_iter': [1000],
            'reg_lambda': [0.001, 0.01, 0.1],
            'fit_intercept': [True]
        }
    
    print("Phase 1: Finding optimal hyperparameters...")
    grid_results = grid_search(
        X_train, y_train, 
        param_grid=param_grid,
        cv=cv,
        n_jobs=n_jobs,
        loss_type="baseline",
        coordinate_choice="random"
    )
    
    best_params = grid_results['best_params']
    print(f"Best parameters found: {best_params}")
    
    # Fix max_iter to a larger value for convergence tracking
    best_params['max_iter'] = 1000
    
    # Phase 2: Compare baseline vs custom loss
    print("\nPhase 2: Comparing baseline vs custom loss with random coordinate selection...")
    baseline_model = LogisticRegression(**best_params)
    baseline_accuracies = baseline_model.fit(X_train, y_train, loss_type="baseline", 
                                           coordinate_choice="random", verbose=False)
    
    custom_model = LogisticRegression(**best_params)
    custom_accuracies = custom_model.fit(X_train, y_train, loss_type="custom", 
                                        coordinate_choice="random", verbose=False)
    
    # Phase 3: Compare all coordinate selection methods
    print("\nPhase 3: Comparing all coordinate selection methods with custom loss...")
    
    # Already have random model from Phase 2
    random_model = custom_model
    random_accuracies = custom_accuracies
    
    # Train models with all coordinate selection methods
    coordinate_methods = ["gradient_based", "cyclic", "cyclic_momentum", 
                         "importance_sampling", "greedy_subset", "adaptive_frequency"]
    
    models = {}
    accuracies = {}
    test_accuracies = {}
    
    # Random and baseline already calculated
    models["random"] = random_model
    accuracies["random"] = random_accuracies
    test_accuracies["random"] = random_model.score(X_test, y_test)
    
    # Train for each coordinate method
    for method in coordinate_methods:
        print(f"  Training with {method} coordinate selection...")
        model = LogisticRegression(**best_params)
        acc = model.fit(X_train, y_train, loss_type="custom", 
                      coordinate_choice=method, verbose=False)
        
        models[method] = model
        accuracies[method] = acc
        test_accuracies[method] = model.score(X_test, y_test)
    
    # Also train baseline model with gradient-based coordinate selection for comparison
    baseline_gradient_model = LogisticRegression(**best_params)
    baseline_gradient_accuracies = baseline_gradient_model.fit(X_train, y_train, loss_type="baseline", coordinate_choice="gradient_based", verbose=False)
    
    # Print test accuracies
    print(f"\nTest accuracies:")
    print(f"Baseline loss with random coord: {baseline_model.score(X_test, y_test):.4f}")
    print(f"Custom loss with random coord: {test_accuracies['random']:.4f}")
    
    for method in coordinate_methods:
        print(f"Custom loss with {method} coord: {test_accuracies[method]:.4f}")
    
    print(f"Baseline loss with gradient-based coord: {baseline_gradient_model.score(X_test, y_test):.4f}")
    
    # Create and save separate figures
    
    # Figure 1: Loss comparison
    plt.figure(figsize=(10, 6))
    iterations = np.arange(len(baseline_accuracies))
    plt.plot(iterations, baseline_accuracies, label='Baseline Loss')
    plt.plot(iterations, custom_accuracies, label='Custom Loss')
    plt.xlabel('Iterations')
    plt.ylabel('Accuracy')
    plt.title('Loss Function Comparison')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('loss_function_comparison.png')
    plt.close()
    
    # Figure 2: Coordinate selection comparison with all methods
    plt.figure(figsize=(12, 8))
    
    # Plot for random method (already calculated)
    plt.plot(iterations, accuracies["random"], label='Random', linewidth=2)
    
    # Colors for better visualization
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']
    
    # Plot each coordinate method
    for i, method in enumerate(coordinate_methods):
        plt.plot(iterations, accuracies[method], label=method.replace('_', ' ').title(), 
                 linewidth=2, color=colors[i % len(colors)])
    
    plt.xlabel('Iterations')
    plt.ylabel('Accuracy')
    plt.title('Comparison of All Coordinate Selection Methods')
    plt.legend(loc='lower right')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('all_coordinate_methods_comparison.png')
    plt.close()
    
    # Figure 3: Coordinate selection with baseline loss comparison
    plt.figure(figsize=(10, 6))
    plt.plot(iterations, baseline_accuracies, label='Baseline Loss - Random')
    plt.plot(iterations, baseline_gradient_accuracies, label='Baseline Loss - Gradient-based')
    plt.xlabel('Iterations')
    plt.ylabel('Accuracy')
    plt.title('Coordinate Selection Comparison with Baseline Loss')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('baseline_coordinate_comparison.png')
    plt.close()
    
    # Create result dictionary
    result = {
        'grid_search_results': grid_results,
        'test_accuracies': {
            'baseline_random': baseline_model.score(X_test, y_test),
            'custom_random': test_accuracies['random'],
        },
        'training_accuracies': {
            'baseline_random': baseline_accuracies,
            'custom_random': random_accuracies,
        }
    }
    
    # Add all coordinate methods to results
    for method in coordinate_methods:
        result['test_accuracies'][f'custom_{method}'] = test_accuracies[method]
        result['training_accuracies'][f'custom_{method}'] = accuracies[method]
    
    # Add baseline with gradient
    result['test_accuracies']['baseline_gradient'] = baseline_gradient_model.score(X_test, y_test)
    result['training_accuracies']['baseline_gradient'] = baseline_gradient_accuracies
    
    return result

In [77]:
# Define parameter grid
param_grid = {
    'lr': [0.01, 0.001],
    'reg_lambda': [0.1, 1.0],
    'max_iter': [100,500,1000]
}

# Perform grid search
results = grid_search(X, y, param_grid, verbose=True, scoring='f1')

# Create model with best parameters
best_model = LogisticRegression(**results['best_params'])
best_model.fit(X_train, y_train)

Parallel execution failed with error: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.
Falling back to sequential execution...
Evaluating parameters: {'lr': 0.01, 'reg_lambda': 0.1, 'max_iter': 100}
Parameters {'lr': 0.01, 'reg_lambda': 0.1, 'max_iter': 100} - Mean f1: 0.6617
Average unique accuracy points: 30.8/100 (higher suggests better convergence)
Evaluating parameters: {'lr': 0.01, 'reg_lambda': 0.1, 'max_iter': 500}
Parameters {'lr': 0.01, 'reg_lambda': 0.1, 'max_iter': 500} - Mean f1: 0.8218
Average unique accuracy points: 23.2/100 (higher suggests better convergence)
Evaluating parameters: {'lr': 0.01, 'reg_lambda': 0.1, 'max_iter': 1000}
Parameters {'lr': 0.01, 'reg_lambda': 0.1, 'max_iter': 1000} - Mean f1: 0.8374
Average unique accuracy points: 20.8/100 (higher suggests better convergence)
Evaluating parameters: {'lr': 0.01, 'reg_lambda': 1.0, 'max_iter': 100}
Parameters {'lr': 0.01, 'reg_lambda': 1.0, 'max_iter': 100} -

array([0.47115385, 0.75961538, 0.61538462, 0.85576923, 0.77884615,
       0.75      , 0.72115385, 0.76923077, 0.76923077, 0.77884615,
       0.79807692, 0.77884615, 0.68269231, 0.78846154, 0.78846154,
       0.79807692, 0.79807692, 0.81730769, 0.81730769, 0.82692308,
       0.79807692, 0.79807692, 0.81730769, 0.79807692, 0.81730769,
       0.81730769, 0.79807692, 0.79807692, 0.79807692, 0.78846154,
       0.80769231, 0.79807692, 0.80769231, 0.78846154, 0.80769231,
       0.79807692, 0.80769231, 0.81730769, 0.78846154, 0.81730769,
       0.81730769, 0.80769231, 0.78846154, 0.76923077, 0.75      ,
       0.73076923, 0.72115385, 0.76923077, 0.76923077, 0.75      ,
       0.76923077, 0.77884615, 0.77884615, 0.77884615, 0.77884615,
       0.78846154, 0.79807692, 0.77884615, 0.76923077, 0.77884615,
       0.79807692, 0.80769231, 0.78846154, 0.78846154, 0.76923077,
       0.77884615, 0.77884615, 0.78846154, 0.78846154, 0.77884615,
       0.76923077, 0.76923077, 0.76923077, 0.76923077, 0.76923

In [78]:
fullBreakdown(X, y, param_grid)

Phase 1: Finding optimal hyperparameters...
Parallel execution failed with error: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.
Falling back to sequential execution...
Evaluating parameters: {'lr': 0.01, 'reg_lambda': 0.1, 'max_iter': 100}
Parameters {'lr': 0.01, 'reg_lambda': 0.1, 'max_iter': 100} - Mean accuracy: 0.8752
Average unique accuracy points: 23.0/100 (higher suggests better convergence)
Evaluating parameters: {'lr': 0.01, 'reg_lambda': 0.1, 'max_iter': 500}
Parameters {'lr': 0.01, 'reg_lambda': 0.1, 'max_iter': 500} - Mean accuracy: 0.9229
Average unique accuracy points: 13.8/100 (higher suggests better convergence)
Evaluating parameters: {'lr': 0.01, 'reg_lambda': 0.1, 'max_iter': 1000}
Parameters {'lr': 0.01, 'reg_lambda': 0.1, 'max_iter': 1000} - Mean accuracy: 0.8752
Average unique accuracy points: 11.6/100 (higher suggests better convergence)
Evaluating parameters: {'lr': 0.01, 'reg_lambda': 1.0, 'max_iter': 100}

{'grid_search_results': {'best_params': {'lr': 0.01,
   'reg_lambda': 0.1,
   'max_iter': 1000},
  'best_score': np.float64(0.9228571428571429),
  'best_convergence': array([0.45430293, 0.58872633, 0.60820425, 0.75246701, 0.8294607 ,
         0.88711991, 0.90886403, 0.89202524, 0.87997705, 0.89911073,
         0.90381526, 0.91095812, 0.91818703, 0.9109868 , 0.90149168,
         0.89176707, 0.88453815, 0.87251865, 0.86537579, 0.87971888,
         0.87257602, 0.87016638, 0.86537579, 0.87983362, 0.87983362,
         0.89905336, 0.8894148 , 0.90625359, 0.90140562, 0.89899598,
         0.90857717, 0.90146299, 0.90863454, 0.89902467, 0.91345382,
         0.91583477, 0.91101549, 0.91583477, 0.91583477, 0.91342513,
         0.91104418, 0.90387263, 0.90387263, 0.90384395, 0.89905336,
         0.9062249 , 0.91342513, 0.91339644, 0.92071142, 0.92071142,
         0.92065404, 0.92306368, 0.91827309, 0.92068273, 0.91113024,
         0.91113024, 0.91353987, 0.9134825 , 0.9087206 , 0.9087206 ,
       

In [79]:
y_pred = best_model.predict(X_test)
accuracy = best_model.score(X_test, y_test)

# Evaluate performance
# accuracy = accuracy_score(y_test, y_pred)
print(f"Custom Baseline Logistic Regression Accuracy: {accuracy}")


Custom Baseline Logistic Regression Accuracy: 1.0


In [80]:
print(y_pred[:20])
print(y_test[:20].values)

[1 2 2 1 1 2 2 2 2 2 2 2 1 1 2 1 2 2 2 1]
[1 2 2 1 1 2 2 2 2 2 2 2 1 1 2 1 2 2 2 1]
